In [11]:
import torch
import torch.nn as nn

In [12]:
%pip install unidecode

Note: you may need to restart the kernel to use updated packages.


In [13]:
import unidecode
import string
import random
import re
import time, math

In [14]:
num_epochs = 2000
print_every = 100
plot_every = 10

# chunk에 대한 설명은 아래 함수정의하면서 하겠습니다.
chunk_len = 200

hidden_size = 100
batch_size = 1
num_layers = 1
embedding_size = 70
lr = 0.002

In [15]:
all_characters = string.printable

n_characters = len(all_characters)
print(all_characters)
print("num_chars = ", n_characters)

0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ 	

num_chars =  100


In [16]:
file = unidecode.unidecode(open("./data/input.txt").read())
file_len = len(file)
print("file_len =", file_len)

file_len = 1115394


In [17]:
# 텍스트 파일의 일부분을 랜덤하게 불러오는 코드
def random_chunk():
    start_index = random.randint(0, file_len - chunk_len)
    end_index = start_index + chunk_len +1
    return file[start_index:end_index]

print(random_chunk())

ce,--

MENENIUS:
Well, well, no more of that.

CORIOLANUS:
Though there the people had more absolute power,
I say, they nourish'd disobedience, fed
The ruin of the state.

BRUTUS:
Why, shall the people


In [18]:
# 문자열을 받았을때 이를 인덱스의 배열로 바꿔주는 함수
def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        tensor[c] = all_characters.index(string[c])
    return tensor

print(char_tensor('ABCdef'))

tensor([36, 37, 38, 13, 14, 15])


In [19]:
# 랜덤한 텍스트 chunk를 불러와서 이를 입력과 목표값을 바꿔주는 함수
# pytorch라는 문자열이 들어오면 입력은 pytorc / 목표값은 ytorch가 된다.
def random_training_set():
    chunk = random_chunk()
    inp = char_tensor(chunk[:-1])
    target = char_tensor(chunk[1:])
    return inp, target

In [20]:
class RNN(nn.Module):
    def __init__(
        self, input_size, embedding_size, hidden_size, output_size, num_layers=1
    ):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.rnn = nn.RNN(self.embedding_size, self.hidden_size, self.num_layers)
        self.decoder = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden):
        out = self.encoder(input.view(1, -1))
        out, hidden = self.rnn(out, hidden)
        out = self.decoder(out.view(batch_size, -1))
        return out, hidden

    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, batch_size, self.hidden_size)
        return hidden


model = RNN(n_characters, embedding_size, hidden_size, n_characters, num_layers)

In [21]:
model = RNN(
    input_size=n_characters,
    embedding_size=embedding_size,
    hidden_size=hidden_size,
    output_size=n_characters,
    num_layers=2,
)

In [22]:
inp = char_tensor("A")
print(inp)
hidden = model.init_hidden()
print(hidden.size())
out, hidden = model(inp, hidden)
print(out.size())

tensor([36])
torch.Size([2, 1, 100])
torch.Size([1, 100])


In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_func = nn.CrossEntropyLoss()

In [24]:
def test():
    start_str = "b"
    inp = char_tensor(start_str)
    hidden = model.init_hidden()
    x = inp
    
    print(start_str,end="")
    for i in range(200):
        output, hidden = model(x, hidden)
        output_dist = output.data.view(-1).div(0.8).exp()
        top_i = torch.multinomial(output_dist, 1)[0]
        predicted_char = all_characters[top_i]
        
        print(predicted_char, end="")
        
        x = char_tensor(predicted_char)

In [26]:
for i in range(num_epochs):
    inp, label = random_training_set()
    hidden = model.init_hidden()
    
    loss = torch.tensor([0]).type(torch.FloatTensor)
    optimizer.zero_grad()
    for j in range(chunk_len-1):
        x=inp[j]
        y_ = label[j].unsqueeze(0).type(torch.LongTensor)
        y, hidden = model(x,hidden)
        loss += loss_func(y,y_)
        
    loss.backward()
    optimizer.step()
    
    if i % 100 == 0:
        print("\n",loss/chunk_len,"\n")
        test()
        print("\n","="*100)


 tensor([2.3685], grad_fn=<DivBackward0>) 

breare, he the yimer h thas procer dor hatis foue on bich thee will darens thao non weay, frosheek ingr it our cavend pther as sor
What, th sour ptay theats? Kof doung dowe are moy thertret mears note 

 tensor([2.1511], grad_fn=<DivBackward0>) 

be not mence come whow the lered pey you that ou eyou, mortse diend nut nom you dand he than' nom doun noce uen the ward fies're, beron the hy nowk
:
Aigt mo.
Wo Woke aw and too to Ipen stanm hend, to 

 tensor([2.1581], grad_fn=<DivBackward0>) 

besies.

Rrilist of will beather this mome nien momruvonge how feale our it ald hou, the warders in wremarrentem, heat Courden earounst our,
And me pepit out thes ghis me thy of hach'st loum cawerdest 

 tensor([2.1161], grad_fn=<DivBackward0>) 

breoters.

KENHIO:
I hattom live,
And shend ast be sorred is the hit ferle of my hen me sies ast thou my noke to lome wither of that bpis heare and loves ke thee but pilp be that momane noss:
-houe com

 tensor([1.